# Import modules and read data

In [1]:
import json
import numpy as np
import pandas as pd
from pipedream_solver.superlink import SuperLink
from pipedream_solver.nsuperlink import nSuperLink
from pipedream_solver.simulation import Simulation

In [2]:
# Specify data path
model_input_path = '../data/storage_test'
data_input_path = '../data/six_pipes'

# Get model components
superjunctions = pd.read_csv(f'{model_input_path}/superjunctions.csv')
superlinks = pd.read_csv(f'{model_input_path}/superlinks.csv')

# Read input data
Q_in = pd.read_csv(f'{data_input_path}/flow_input.csv', index_col=0)
H_bc = pd.read_csv(f'{data_input_path}/boundary_stage.csv', index_col=0)

# Get storages
with open(f'{model_input_path}/storage.json', 'r') as fp:
    storages = json.load(fp)

In [3]:
# View storage depth-area relationship for one storage element
print(json.dumps(storages['s1'], indent=4))

{
    "h": [
        0,
        0.2,
        0.5,
        0.8,
        1,
        2
    ],
    "A": [
        1e-05,
        1.0,
        1.2,
        1.8,
        2.0,
        3.0
    ]
}


# Create and run hydraulic model

In [4]:
superlink = SuperLink(superlinks, superjunctions, storages=storages)    

In [5]:
# Set initial timestep
dt = 30

# Create simulation context manager
with Simulation(superlink, Q_in=Q_in, H_bc=H_bc) as simulation:
    # While simulation time has not expired...
    for step in simulation.steps:
        if simulation.t >= simulation.t_end:
            break
        # Step model forward in time
        simulation.step(dt=dt)
        # Record internal depth and flow states
        simulation.record_state()
        # Print progress bar
        simulation.print_progress()

[=================================================-] 98.1% [4.37 s]

# Plot results

In [6]:
# Plot results
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('husl')
%matplotlib inline

# Instantiate plot
fig, ax = plt.subplots(3, figsize=(10, 12))

# Compute average discharge in superlinks
simulation.states.Q_k = (simulation.states.Q_uk + simulation.states.Q_dk) / 2

# Plot results
simulation.Q_in[['A', 'C']].plot(ax=ax[0], title='Input discharge (cms)')
simulation.H_bc[['D', 'F']].plot(ax=ax[1], title='Boundary stage (m)')
simulation.states.Q_k.plot(ax=ax[2], title='Superlink discharge (cms)')

# Configure plots
ax[0].set_ylabel('Input discharge (cms)')
ax[1].set_ylabel('Boundary stage (m)')
ax[2].set_ylabel('Superlink discharge (cms)')
plt.tight_layout()

# Test accelerated solver

In [7]:
superlink = nSuperLink(superlinks, superjunctions, storages=storages)    

In [8]:
# Set initial timestep
dt = 30

# Create simulation context manager
with Simulation(superlink, Q_in=Q_in, H_bc=H_bc) as simulation:
    # While simulation time has not expired...
    for step in simulation.steps:
        if simulation.t >= simulation.t_end:
            break
        # Step model forward in time
        simulation.step(dt=dt)
        # Record internal depth and flow states
        simulation.record_state()
        # Print progress bar
        simulation.print_progress()

[=================================================-] 98.1% [0.93 s]

In [9]:
# Instantiate plot
fig, ax = plt.subplots(3, figsize=(10, 12))

# Compute average discharge in superlinks
simulation.states.Q_k = (simulation.states.Q_uk + simulation.states.Q_dk) / 2

# Plot results
simulation.Q_in[['A', 'C']].plot(ax=ax[0], title='Input discharge (cms)')
simulation.H_bc[['D', 'F']].plot(ax=ax[1], title='Boundary stage (m)')
simulation.states.Q_k.plot(ax=ax[2], title='Superlink discharge (cms)')

# Configure plots
ax[0].set_ylabel('Input discharge (cms)')
ax[1].set_ylabel('Boundary stage (m)')
ax[2].set_ylabel('Superlink discharge (cms)')
plt.tight_layout()